In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
%matplotlib widget

import sys
sys.path.append('/home/dtward/data/csh_data/emlddmm')
import emlddmm
import csv
from skimage.measure import marching_cubes
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from os.path import split,join,splitext

In [ ]:
ontology_name = '/nafs/dtward/dong/upenn_atlas/atlas_info_KimRef_FPbasedLabel_v2.7.csv'
seg_name = '/nafs/dtward/dong/upenn_atlas/UPenn_labels_reoriented_origin.vtk'

### Load UPenn ontology + Generate lists of descendents

In [ ]:
xS,S,_,_ = emlddmm.read_data(seg_name)

In [ ]:
parent_column = 7 # 8 for allen, 7 for yongsoo
label_column = 0 # 0 for both
shortname_column = 2# 3 for allen, 2 for yongsoo
longname_column = 1# 2 for allen, 1 for yongsoo
ontology = dict()
with open(ontology_name) as f:
    csvreader = csv.reader(f, delimiter=',', quotechar='"')
    count = 0
    for row in csvreader:        
        if count == 0:
            headers = row
            print(headers)
        else:
            if not row[parent_column]:
                parent = -1
            else:
                parent = int(row[parent_column])
            ontology[int(row[label_column])] = (row[shortname_column],row[longname_column],parent)
        count += 1


In [ ]:
# we need to find all the descendants of a given label
# first we'll get children
children = dict()
for o in ontology:
    parent = ontology[o][-1]
    if parent not in children:
        children[parent] = []
    children[parent].append(o)

In [ ]:
# now we go from children to descendents
descendents = dict(children)
for o in descendents:
    for child in descendents[o]:
        if child in descendents: # if I don't do this i get a key error 0
            descendents[o].extend(descendents[child])
descendents[0] = []

In [ ]:
descendents_and_self = dict(descendents)
for o in ontology:
    if o not in descendents_and_self:
        descendents_and_self[o] = [o]
    else:
        descendents_and_self[o].append(o)

In [ ]:
ontology

## Generate Boolean masks for CP, CPr (+CPre), CPi, CPc (+CPce)

In [ ]:
# find all the descendents of Caudoputemen- rostral
# and caudoputemen rostral- extreme

In [ ]:
# these are all the structures in the CP
cp = list(descendents_and_self[672])
# get caudate
Scp = np.zeros_like(S)
for l in cp:
    Scp = np.logical_or(Scp,S==l)

In [ ]:
# these are the structures one level below CP, with extreme's merged in
rostral = list(descendents_and_self[2376]) # rostral extreme
rostral.extend(list(descendents_and_self[2491])) # rostral 
rostral = list(dict.fromkeys(rostral))

intermediate = list(descendents_and_self[2492]) # intermediate

caudal = list(descendents_and_self[2496]) # caudal
caudal_ = list(descendents_and_self[2495]) # caudal extreme
caudal.extend(caudal_)

In [ ]:
Srostral = np.zeros_like(S)
for l in rostral:
    Srostral = np.logical_or(Srostral, S==l)

In [ ]:
Sintermediate = np.zeros_like(S)
for l in intermediate:
    Sintermediate = np.logical_or(Sintermediate, S==l)

In [ ]:
Scaudal = np.zeros_like(S)
for l in caudal:
    Scaudal = np.logical_or(Scaudal, S==l)

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.sum(Srostral>0,axis=(0,2,3)),label='r')
ax.plot(np.sum(Sintermediate>0,axis=(0,2,3)),label='i')
ax.plot(np.sum(Scaudal>0,axis=(0,2,3)),label='c')
ax.legend()

In [ ]:
# TODO
# add one more level down the tree

In [ ]:
S.shape

In [ ]:
# I think what I'd like to do is assign a gaussian to each region
# then give the neurons a distribution based on the Gaussian
# to do this I should load a set of neurons
# and also start visualizing

In [ ]:
# what I think would make sense is to take all the cp structures
# blur them a lot
# then assign probabilities
# we also want to look at the neurons though


In [ ]:
# i want to start by visualizing
# we need to load swc files
# and we need to contour the surfaces
d = [x[1] - x[0] for x in xS]


In [ ]:
down = 16
xSd,Scpd = emlddmm.downmode(xS,Scp[0],down=[down,down,down])

In [ ]:
dd = [x[1] - x[0] for x in xSd]

In [ ]:
verts,faces,normals,values = marching_cubes(Scpd,level=0.5,spacing=dd)
verts = verts + np.array([x[0] for x in xSd])

## Load SWC files and Display 2x2 figure

In [ ]:
# swcdir = '../swc_out_v08' # this is tme07
# swcdir = '../dragonfly_tme09-1/swc_out_v08'
# swcdir = '/home/abenneck/dragonfly_work/dragonfly_outputs/TME08-1/dragonfly_joint_outputs'

brain = 'TME12-1'
swcdir = f'/home/abenneck/dragonfly_work/dragonfly_outputs/{brain}/dragonfly_joint_outputs'
files = glob(join(swcdir,'*.swc'))
files = [f for f in files if 'permuted' not in f]
files

In [ ]:
x = []
for file in files:
    with open(file)  as f:
        for i,line in enumerate(f):    
            print(line)
            if 'Tward' in line:                
                continue
            else:
                if ',' in line:
                    lim = ','
                else:
                    lim = ''
                coords = [float(c) for c in line.split(lim)[2:5]]                
                    
                x.append(coords)
                break
    
            
x = np.array(x)            
            

### Generate + Save figure

In [ ]:
s = 5
alpha = 0.25
lw = 0.25


mesh = Poly3DCollection(verts[faces],ec=[0.0,0.0,0.0,0.1],lw=lw,alpha=alpha,)
fig = plt.figure()
ax = fig.add_subplot(2,2,1,projection='3d')
ax.add_collection3d(mesh)
# set limits uniform
vertsmin = np.min(verts,0)
vertsmax = np.max(verts,0)
vertsc = vertsmin*0.5 + vertsmax*0.5
vertsd = vertsmax-vertsmin
vertsd = np.max(vertsd)
lim = vertsc[None] + np.array([-1,1])[...,None]/2*vertsd
ax.set_xlim(lim[:,0])
ax.set_ylim(lim[:,1])
ax.set_zlim(lim[:,2])
ax.set_xlabel('x0')
ax.set_ylabel('x1')
ax.set_zlabel('x2')
ax.scatter(x[:,0],x[:,1],x[:,2],s=s)


ax = fig.add_subplot(2,2,2,projection='3d')
mesh = Poly3DCollection(verts[faces],ec=[0.0,0.0,0.0,0.1],lw=lw,alpha=alpha,)
ax.view_init(0,90)
ax.add_collection3d(mesh)
# set limits uniform
vertsmin = np.min(verts,0)
vertsmax = np.max(verts,0)
vertsc = vertsmin*0.5 + vertsmax*0.5
vertsd = vertsmax-vertsmin
vertsd = np.max(vertsd)
lim = vertsc[None] + np.array([-1,1])[...,None]/2*vertsd
ax.set_xlim(lim[:,0])
ax.set_ylim(lim[:,1])
ax.set_zlim(lim[:,2])
ax.set_xlabel('x0')
ax.set_ylabel('x1')
ax.set_zlabel('x2')
ax.scatter(x[:,0],x[:,1],x[:,2],s=s)



ax = fig.add_subplot(2,2,3,projection='3d')
mesh = Poly3DCollection(verts[faces],ec=[0.0,0.0,0.0,0.1],lw=lw,alpha=alpha,)
ax.view_init(0,0)
ax.add_collection3d(mesh)
# set limits uniform
vertsmin = np.min(verts,0)
vertsmax = np.max(verts,0)
vertsc = vertsmin*0.5 + vertsmax*0.5
vertsd = vertsmax-vertsmin
vertsd = np.max(vertsd)
lim = vertsc[None] + np.array([-1,1])[...,None]/2*vertsd
ax.set_xlim(lim[:,0])
ax.set_ylim(lim[:,1])
ax.set_zlim(lim[:,2])
ax.set_xlabel('x0')
ax.set_ylabel('x1')
ax.set_zlabel('x2')
ax.scatter(x[:,0],x[:,1],x[:,2],s=s)



ax = fig.add_subplot(2,2,4,projection='3d')
mesh = Poly3DCollection(verts[faces],ec=[0.0,0.0,0.0,0.1],lw=lw,alpha=alpha,)
ax.view_init(90,0)
ax.add_collection3d(mesh)
# set limits uniform
vertsmin = np.min(verts,0)
vertsmax = np.max(verts,0)
vertsc = vertsmin*0.5 + vertsmax*0.5
vertsd = vertsmax-vertsmin
vertsd = np.max(vertsd)
lim = vertsc[None] + np.array([-1,1])[...,None]/2*vertsd
ax.set_xlim(lim[:,0])
ax.set_ylim(lim[:,1])
ax.set_zlim(lim[:,2])
ax.set_xlabel('x0')
ax.set_ylabel('x1')
ax.set_zlabel('x2')
ax.scatter(x[:,0],x[:,1],x[:,2],s=s)

# fig.suptitle(swcdir.split('/')[-2])
# fig.savefig('CP_SWC_QC_figure_'+swcdir.split('/')[-2]+'.jpg')

fig.suptitle(f'{brain}')
fig.savefig(join(f'/home/abenneck/dragonfly_work/dragonfly_outputs/{brain}/',f'CP_SWC_QC_figure_{brain}.jpg'))

In [ ]:
# raise Exception(f'End of QC figure generation for {brain}')

In [ ]:
# so what is a plan going forward
# I'd like to take every blob as a gaussian
# the height is its volume
# the mean is its mean
# the covariance is its covariance
# the only trouble here is the left right issue
# another posiblility is to just blur the labels
# this may get rid of small structures though
# but I could give less blur to the small structures

In [ ]:
# note, xS[1] should be left right

# Below line was ran in original version
# xS[1]>0

In [ ]:
# I can just make positive x1 to do this

# Below lines were ran in original version
# Srostral
# Scaudal
# Sintermediate

In [ ]:
# first going forward
# 1. make a figure like this to get a sense of the uncertainty for each brain
# 2. make a probabilitistic version of the CP structures.
# I'd like to model each structure as a gaussian blob (ellispoids with soft boundaries)
# this needs three parameters
# the mean (a 3 element vector)
# the covariance (3x3 symmetric matrix)
# and the height/amplitude of the gaussian (one positive number)

# The height (amplitude) should be the number of voxels in the structure
# the mean, is going to be the first moment of the segmentation
# the covariance, is the second central moment of the segmentation.

### Compute + Save Gaussian parameters for CP

In [ ]:
# Save original Scp since variable may be updated in below cell
Scp_ = Scp

In [ ]:
updateFile = False
# ===== Specify subregion and hemisphere =====

# Scp = Scp_
# Scp = Srostral
# Scp = Sintermediate
# Scp = Scaudal

hemi = 'L'
if hemi == 'L':
    LR_indicator = (xS[1][:,None]>=0) # Left hemisphere?
else:
    LR_indicator = (xS[1][:,None]<0)  # Right hemisphere?

# ===== Compute Gaussian parameters =====

# number of voxels in CP
Ncp = np.sum(Scp*LR_indicator)
print(f'number of voxels {Ncp}')

# note xS tells us the location of each voxel
# let's compute the first moment
mucp = [np.sum(xS[0][:,None,None]*Scp*LR_indicator)/Ncp, np.sum(xS[1][:,None]*Scp*LR_indicator)/Ncp, np.sum(xS[2][:]*Scp*LR_indicator)/Ncp]
print(f'mu {mucp}')

# now calculate the covariance matrix
covcp01 = np.sum((xS[0][:,None,None] - mucp[0])*(xS[1][:,None] - mucp[1])*Scp*LR_indicator)/Ncp # here row 0 column 1
covcp02 = np.sum((xS[0][:,None,None] - mucp[0])*(xS[2][None,:] - mucp[2])*Scp*LR_indicator)/Ncp # here row 0 column 2
covcp12 = np.sum((xS[1][None,:,None] - mucp[1])*(xS[2][None,:] - mucp[2])*Scp*LR_indicator)/Ncp # here row 1 column 2

covcp00 = np.sum(((xS[0][:,None,None] - mucp[0])**2)*Scp*LR_indicator)/Ncp # here row 0 column 0
covcp11 = np.sum(((xS[1][None,:,None] - mucp[1])**2)*Scp*LR_indicator)/Ncp # here row 1 column 1
covcp22 = np.sum(((xS[2][None,None,:] - mucp[2])**2)*Scp*LR_indicator)/Ncp # here row 2 column 2

covcp = [[covcp00,covcp01,covcp02],[covcp01,covcp11,covcp12],[covcp02,covcp12,covcp22]]

print('cov:')
print(f'{covcp[0]}')
print(f'{covcp[1]}')
print(f'{covcp[2]}')

# Save Gaussian parameters in.npz file with keys [height, mu, cov]
if updateFile:
    fname = f'cpc_param_{hemi}.npz'
    save_path = join('/home/abenneck/dragonfly_work/gaussian_parameters/',fname)
    np.savez(save_path,height=[Ncp],mu=mucp,cov=covcp)

### Load parameters given hemi and subregion

In [ ]:
hemi = 'L'
subregion = ''

fname = f'cp{subregion}_param_{hemi}.npz'
fpath = join(f'/home/abenneck/dragonfly_work/gaussian_parameters/',fname)
out = np.load(fpath)

h = out['height'][0]
mu = out['mu']
cov = out['cov']

print(f'Displating paremeters for {fname};\n')
print(f'height: {h}')
print(f'mu: {mu}')
print(f'cov: {cov}')

In [ ]:
# once we have parameters for each gaussian
# then we can evalute them at every point in space
# so for a given cell body, we can find a distribution over this number of regions
# for each level of the tree, you can get a distribution over all the structures in that level
# start with R-I-C level
# TODO, come up with some measure of concordance
#   that is related to nick's labels, but allows some slop

In [ ]:
# we don't really believe the uncertaint is given by the voxel size (we're not getting 1 voxel accurate registration)
# we believe it is related to the anatomy

In [ ]:
import pandas as pd
from scipy.stats import multivariate_normal
import os

# Define relevant directories
brain = 'TME08-1'
neuronDir = f'/home/abenneck/dragonfly_work/dragonfly_outputs/{brain}/dragonfly_joint_outputs/'
paramDir = '/home/abenneck/dragonfly_work/gaussian_parameters'
allDist = list()
allProb = pd.DataFrame(columns=['name','CP_L','CP_R','CPc_L','CPc_R','CPi_L','CPi_R','CPr_L','CPr_R'])

# Create list of multivariate Gaussian RVs from presaved parameters
for file in sorted(os.listdir(paramDir)[1:]):
    data = np.load(os.path.join(paramDir,file))
    norm_factor = data['height'].item()
    mu = data['mu']
    cov = data['cov']
    allDist.append([multivariate_normal(mean=mu, cov=cov, allow_singular=False),norm_factor])

# Generate regional probabilities for every neuron in neuronDir
row_idx = 0
for file in sorted(os.listdir(neuronDir)):
    if "permuted.swc" in file:
        # Load soma coordinates
        data = pd.read_csv(os.path.join(neuronDir,file))
        x = data.columns[2]
        y = data.columns[3]
        z = data.columns[4]
        soma_location = [x,y,z]

        # Compute probabilities form multinomial Gaussians
        neuronProb = [file]
        for dist in allDist:
            neuronProb.append(dist[0].pdf(soma_location)*dist[1])

        # Normalize probabilities at different scales
        neuronProb[1:3] = neuronProb[1:3] / np.sum(neuronProb[1:3]) # CP (L+R)
        neuronProb[3:] = neuronProb[3:] / np.sum(neuronProb[3:])    # CPc, CPi, CPr (L+R)
        neuronProb[1:] = [f'{x:.3e}' for x in neuronProb[1:]]       # 4 sig figs
        
        # neuronProb.insert(0,file)
        allProb.loc[row_idx] = neuronProb
        row_idx+=1

tempDir = f'/home/abenneck/dragonfly_work/{brain}_neuron_region_prob.csv'

allProb.to_csv(tempDir, index=False)

# allProb

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
val = 0.000000000000123456789

print(f'{val:.3e}')